In [22]:
import os
import struct
import numpy as np
from PIL import Image
import subprocess
from osgeo import gdal; gdal.UseExceptions()
import warnings; warnings.filterwarnings("ignore")
from p_tqdm import p_umap
from util import recreate_dir, load_config
import shutil

In [23]:
DATA_DIR = "data/20221220_sudol_krakow"
CFG = load_config(f"{DATA_DIR}/config.py").CONVERSION

In [24]:
RJPG_DIR = f"{DATA_DIR}/rjpg"
TIF_RAW_DIR = f"{DATA_DIR}/tif_raw"
TMP_DIR = f"{DATA_DIR}/tmp"

# R-JPG to TIF

In [25]:
distance = round(CFG.DISTANCE)# DJI Thermal SDK requires integer values
humidity = round(CFG.HUMIDITY)# DJI Thermal SDK requires integer values
emissivity = round(CFG.EMISSIVITY,2)# DJI Thermal SDK requires float values with 2 decimal places
reflection = round(CFG.REFLECTION)# DJI Thermal SDK requires integer values
assert os.path.exists(RJPG_DIR)
recreate_dir(TIF_RAW_DIR)
recreate_dir(TMP_DIR)

'data/20221220_sudol_krakow/tmp'

In [26]:
def rjpg2tif(file_name):
    try:
        #Run DJI Thermal SDK conversion to temperature byte file
        subprocess.check_call(['./dji_irp', '-a', 'measure', '--measurefmt', 'float32', 
                        '--distance', str(distance), '--humidity', str(humidity), 
                        '--emissivity', str(emissivity), '--reflection', str(reflection), 
                        '-s', f"{RJPG_DIR}/{file_name}", '-o', f"{TMP_DIR}/{file_name}"], 
                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError as e:
        return False
    img = Image.open(f"{RJPG_DIR}/{file_name}")
    # Decode temporary byte file to tif
    arr = np.zeros(img.size[0]*img.size[1])
    with open(f"{TMP_DIR}/{file_name}", "rb") as f:
        data = f.read()
        format = '{:d}f'.format(len(data)//4)
        arr=np.array(struct.unpack(format, data))
    arr = arr.reshape(img.size[1],img.size[0])
    im = Image.fromarray(arr)
    tif_file_name = file_name.replace('.JPG','.tif')
    im.save(f"{TIF_RAW_DIR}/{tif_file_name}")
    # copy exif data from RJPG file to new tif
    subprocess.check_call(['exiftool', '-tagsfromfile', f"{RJPG_DIR}/{file_name}", f"{TIF_RAW_DIR}/{tif_file_name}", '-overwrite_original_in_place'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return True


In [27]:
results = p_umap(rjpg2tif, os.listdir(RJPG_DIR))
print(f"Successfully converted {sum(results)} files out of {len(results)}")

  0%|          | 0/469 [00:00<?, ?it/s]

Successfully converted 469 files out of 469


# Cleanup

In [28]:
#clear temporary files
shutil.rmtree(TMP_DIR)